In [ ]:
import math
import torch


## Задание 1

Напишите функцию, которая моделирует один нейрон с сигмоидной активацией и реализует вычисление градиента для обновления весов и смещений нейрона. Функция должна принимать список векторов признаков, ассоциированные бинарные метки класса, начальные веса, начальное смещение, скорость обучения и количество эпох. Функция должна обновлять веса и смещение с помощью градиентного спуска (классической версии) на основе функции потерь NLL и возвращать обновленные веса, смещение и список значений NLL для каждой эпохи, округленное до четырех десятичных знаков. Проведите обучение на предоставленном наборе данных из задания 4 (для двух разных лет). Опционально сгенерируйте другие подходящие наборы данных. Опишите ваши результаты. Предоставленная функция будет также протестирована во время защиты ДЗ. Можно использовать только чистый torch (без использования autograd и torch.nn).

In [ ]:
def sigmoid(z: torch.Tensor) -> torch.Tensor:
    # sigmoid(z) = 1 / (1 + exp(-z))
    return torch.where(z >= 0, 1 / (1 + torch.exp(-z)), torch.exp(z) / (1 + torch.exp(z)))


def binary_nll(probs: torch.Tensor, y: torch.Tensor, eps: float = 1e-12) -> torch.Tensor:
    probs = torch.clamp(probs, eps, 1 - eps)
    return -(y * torch.log(probs) + (1 - y) * torch.log(1 - probs)).mean()


features: список списков/тензор размера [N, D]    
labels:   список/тензор размера [N], значения {0,1}   
initial_weights: список/тензор [D]    
nitial_bias: float    
learning_rate: float    
epochs: int   

In [ ]:
def train_single_sigmoid_neuron_sgd(
    features,
    labels,
    initial_weights,
    initial_bias: float,
    learning_rate: float,
    epochs: int,
):


    X = torch.tensor(features, dtype=torch.float32)
    y = torch.tensor(labels, dtype=torch.float32)

    w = torch.tensor(initial_weights, dtype=torch.float32).clone()
    b = float(initial_bias)

    nll_history = []

    N = X.shape[0]

    for _ in range(int(epochs)):
        # forward
        z = X @ w + b
        p = sigmoid(z)
        loss = binary_nll(p, y)

        # grad_w = X^T (p - y) / N, grad_b = mean(p - y)
        err = (p - y)
        grad_w = (X.T @ err) / N
        grad_b = err.mean().item()

        # обновялем
        w = w - learning_rate * grad_w
        b = b - learning_rate * grad_b

        nll_history.append(round(loss.item(), 4))

    return w.tolist(), float(b), nll_history


## Задание 3

Реализуйте один из оптимизаторов на выбор. Придумайте и напишите тесты для проверки выбранного оптимизатора. Проведите обучение нейрона из первого задания с использованием оптимизатора, а не ванильного градиентного спуска. Также опишите идею алгоритма (+1 балл).

In [ ]:
class Adagrad:
    def __init__(self, lr: float = 0.1, eps: float = 1e-10):
        self.lr = float(lr)
        self.eps = float(eps)
        self.accum = None
        # будет tensor той же формы, что и параметр

    def step(self, param: torch.Tensor, grad: torch.Tensor) -> torch.Tensor:
        # Возвращает обновленный параметр (не in-place)
        if self.accum is None:
            self.accum = torch.zeros_like(param)

        self.accum = self.accum + grad * grad
        adjusted_lr = self.lr / (torch.sqrt(self.accum) + self.eps)
        return param - adjusted_lr * grad


### Идея алгоритма
Адаптивно подбирает шаг обучения для каждого параметра отдельно на основе истории его градиентов. Параметры с часто наблюдаются большими градиентами получают меньший эффективный шаг, а параметры с малыми градиентами больший шаг.
Шаг монотонно убывает со временем из-за этого обучение может преждевременно тухнуть.

### Тесты для Adagrad
Тестируем:
1) Правильность первого шага (accum = g^2, масштабирование)
2) Обработка разных форм (вектор параметров)


In [ ]:
def test_adagrad_single_step():
    opt = Adagrad(lr=1.0, eps=0.0)
    p0 = torch.tensor([1.0])
    g = torch.tensor([2.0])
    # accum = 4, sqrt=2, lr/sqrt=0.5 => p1 = 1 - 0.5*2 = 0
    p1 = opt.step(p0, g)
    assert torch.allclose(p1, torch.tensor([0.0])), (p1, opt.accum)


def test_adagrad_vector_params():
    opt = Adagrad(lr=1.0, eps=0.0)
    p0 = torch.tensor([1.0, 2.0])
    g = torch.tensor([1.0, 2.0])
    p1 = opt.step(p0, g)
    # coord0: accum=1 sqrt=1 lr=1 => p=0
    # coord1: accum=4 sqrt=2 lr=0.5 => p=2-0.5*2=1
    assert torch.allclose(p1, torch.tensor([0.0, 1.0])), (p1, opt.accum)

test_adagrad_single_step()
test_adagrad_vector_params()
print("Adagrad tests: OK")


Adagrad tests: OK


### Обучение нейрона из задания 1 с Adagrad
Вместо vanilla GD используем оптимизатор для `w` и отдельный оптимизатор для `b` (скаляр).


In [ ]:
def train_single_sigmoid_neuron_adagrad(
    features,
    labels,
    initial_weights,
    initial_bias: float,
    learning_rate: float,
    epochs: int,
    eps: float = 1e-10,
):
    X = torch.tensor(features, dtype=torch.float32)
    y = torch.tensor(labels, dtype=torch.float32)

    w = torch.tensor(initial_weights, dtype=torch.float32).clone()
    b = torch.tensor([float(initial_bias)], dtype=torch.float32)

    opt_w = Adagrad(lr=learning_rate, eps=eps)
    opt_b = Adagrad(lr=learning_rate, eps=eps)

    N = X.shape[0]
    nll_history = []

    for _ in range(int(epochs)):
        z = X @ w + b.item()
        p = sigmoid(z)
        loss = binary_nll(p, y)

        err = (p - y)
        grad_w = (X.T @ err) / N
        grad_b = torch.tensor([err.mean().item()], dtype=torch.float32)  # [1]

        w = opt_w.step(w, grad_w)
        b = opt_b.step(b, grad_b)

        nll_history.append(round(loss.item(), 4))

    return w.tolist(), float(b.item()), nll_history


### Тест

In [ ]:
import pandas as pd
import torch

x_pat = "/content/train_x.csv"
y_path = "/content/train_y.csv"

dfx = pd.read_csv(x_pa)
dfy = pd.read_csv(y_path)

if "Unnamed: 0" in dfx.columns and "Unnamed: 0" in dfy.columns:
    df = dfx.merge(dfy, on="Unnamed: 0", how="inner")
else:
    df = dfx.copy()
    df["year"] = dfy["year"].values

print("merged shape:", df.shape)
print("year unique:", df["year"].nunique(), "range:", int(df["year"].min()), "-", int(df["year"].max()))
print("top years:", df["year"].value_counts().head(10))


merged shape: (14000, 92)
year unique: 79 range: 1922 - 2011
top years:
 year
2007    1102
2006    1031
2005    1001
2008     951
2009     835
2004     792
2003     703
2002     581
2001     580
2000     539
Name: count, dtype: int64


In [ ]:
def make_binary_year_dataset(df: pd.DataFrame, year_pos: int, year_neg: int, normalize: bool = True):
    # 2 года
    sub = df[(df["year"] == year_pos) | (df["year"] == year_neg)].copy()

    # Признаки: все колонки которые выглядят как числа
    drop_cols = set(["year"])

    feature_cols = [c for c in sub.columns if c not in drop_cols]

    X = sub[feature_cols].astype("float32").values
    y = (sub["year"].astype("int64").values == int(year_pos)).astype("int64")  # 1 для year_pos, 0 для year_neg

    if normalize:
        # стандартизация по выборке
        mu = X.mean(axis=0, keepdims=True)
        std = X.std(axis=0, keepdims=True)
        std[std < 1e-6] = 1.0
        X = (X - mu) / std

    return X.tolist(), y.tolist(), feature_cols, sub

# самые частые года
top2 = df["year"].value_counts().head(2).index.tolist()
YEAR_POS, YEAR_NEG = int(top2[0]), int(top2[1])
print("Using years:", YEAR_POS, "(label=1) vs", YEAR_NEG, "(label=0)")

X_bin, y_bin, feature_cols, sub_df = make_binary_year_dataset(df, YEAR_POS, YEAR_NEG, normalize=True)
print("Binary dataset:", len(X_bin), "samples,", len(feature_cols), "features; positive rate:", sum(y_bin)/len(y_bin))


Using years: 2007 (label=1) vs 2006 (label=0)
Binary dataset: 2133 samples, 90 features; positive rate: 0.516643225503985


### Обучение (vanilla GD из задания 1)

In [ ]:
D = len(feature_cols)
init_w = [0.0] * D
init_b = 0.0

w_gd, b_gd, nll_gd = train_single_sigmoid_neuron_sgd(
    X_bin, y_bin,
    initial_weights=init_w,
    initial_bias=init_b,
    learning_rate=0.1,
    epochs=300
)

print("Final NLL (GD):", nll_gd[-1])
print("First 10 NLL:", nll_gd[:10])


Final NLL (GD): 0.6763
First 10 NLL: [0.6931, 0.6923, 0.6915, 0.6909, 0.6903, 0.6898, 0.6893, 0.6889, 0.6884, 0.688]


### Обучение (Adagrad из задания 3)

In [ ]:
w_ada, b_ada, nll_ada = train_single_sigmoid_neuron_adagrad(
    X_bin, y_bin,
    initial_weights=init_w,
    initial_bias=init_b,
    learning_rate=0.5,
    epochs=300,
    eps=1e-10
)

print("Final NLL (Adagrad):", nll_ada[-1])
print("First 10 NLL:", nll_ada[:10])


Final NLL (Adagrad): 0.6765
First 10 NLL: [0.6931, nan, nan, nan, 1.4571, nan, inf, nan, 1.0312, nan]


### Дополнительное
(10 баллов) Реализуйте базовые функции autograd. Можете вдохновиться видео от Andrej Karpathy. Напишите класс, аналогичный предоставленному классу 'Element', который реализует основные операции autograd: сложение, умножение и активацию ReLU. Класс должен обрабатывать скалярные объекты и правильно вычислять градиенты для этих операций посредством автоматического дифференцирования. Плюсом будет набор предоставленных тестов, оценивающих правильность вычислений. Большим плюсом будет, если тесты будут написаны с помощью unittest. Можно использовать только чистый torch (без использования autograd и torch.nn). За каждую нереализованную операцию будет вычитаться 3 балла. Пример: a = Node(2) b = Node(-3) c = Node(10) d = a + b * c e = d.relu() e.backward() print(a, b, c, d, e) Output:  Node(data=2, grad=0)  Node(data=-3, grad=10)  Node(data=10, grad=-3)  Node(data=-28, grad=1)  Node(data=0, grad=1)

### Как должно работать
Запускает обратное распространение градиента по графу.
        1) Строим топологический порядок узлов (DFS).   
        2) Инициализируем grad у текущего узла как 1 (dL/dL).   
        3) Идём по узлам в обратном топологическом порядке и вызываем _backward ().   

In [2]:
import torch

class Node:
    def __init__(self, data, _children=(), _op=''):
        # Приводим к float (можно передавать int/float/torch scalar)
        if isinstance(data, torch.Tensor):
            # Допускаем torch scalar (0-dim), но без autograd
            self.data = float(data.item())
        else:
            self.data = float(data)

        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Node(data={self.data:g}, grad={self.grad:g})"

    # приводим число/Node к Node
    @staticmethod
    def _to_node(x):
        return x if isinstance(x, Node) else Node(x)

    # сложение
    def __add__(self, other):
        other = Node._to_node(other)
        out = Node(self.data + other.data, (self, other), _op='+')

        def _backward():
            # d(out)/d(self)=1, d(out)/d(other)=1
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward = _backward
        return out

    def __radd__(self, other):
        # other + self
        return Node._to_node(other) + self

    # умножение
    def __mul__(self, other):
        other = Node._to_node(other)
        out = Node(self.data * other.data, (self, other), _op='*')

        def _backward():
            # out = self * other
            # d(out)/d(self)=other.data, d(out)/d(other)=self.data
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def __rmul__(self, other):
        # other * self
        return Node._to_node(other) * self

    def relu(self):
        out = Node(self.data if self.data > 0 else 0.0, (self,), _op='relu')

        def _backward():
            # dReLU(x)/dx = 1, если x>0, иначе 0
            self.grad += (1.0 if self.data > 0 else 0.0) * out.grad

        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)

        # Обнуляем грады во всём графе (чтобы повторные backward были корректны)
        for v in topo:
            v.grad = 0.0

        # dL/dL = 1
        self.grad = 1.0

        # В обратном порядке вызываем локальные backward
        for v in reversed(topo):
            v._backward()




In [3]:
a = Node(2)
b = Node(-3)
c = Node(10)
d = a + b * c
e = d.relu()
e.backward()
print(a, b, c, d, e)

CASE 1 (ReLU off):
a b c d e = Node(data=2, grad=0) Node(data=-3, grad=0) Node(data=10, grad=0) Node(data=-28, grad=0) Node(data=0, grad=1)


In [7]:
a = Node(2)
b = Node(-3)
c = Node(-10)
d = a + b * c
e = d.relu()
e.backward()
print(a, b, c, d, e)

Node(data=2, grad=1) Node(data=-3, grad=-10) Node(data=-10, grad=-3) Node(data=32, grad=1) Node(data=32, grad=1)
